### Notes
- use cath as becnhmarking
- eg with pdb structure u get 85%, alphafold 84%, that's good 
- something like an upper limit
- use ./SWORD2.py -p 1jx4 -o results instead of downloading pdb files

In [24]:
from pprint import pprint
import numpy as np

In [2]:
def update_dict(data, code, chain, domain, boundaries):
    data[code] = data.get(code, {})
    if chain not in data[code].keys():
        data[code][chain] = {}
    data[code][chain][str(int(domain))] = boundaries
    return data


def get_cath(lines_range = None, file="../data/cath/cath_domain_boundaries.txt", verb=False):
    with open(file, "r") as f:
        lines = f.readlines() if lines_range is None else f.readlines()[lines_range[0]:lines_range[1]]
        lines = map(lambda x : x.strip(), lines)
        big = 0
        data = {}
        for line in lines:
            name, boundaries = line.split("\t")
            code = name[:4]
            chain = name[4]
            domain = name[5:]
            data = update_dict(data, code, chain, domain, boundaries)
    verb and pprint(data)
    return data


def get_sword2(code, chain, verb=False):
    
    file = f"../data/sword2/SWORD2/results/{code}_{chain.upper()}/sword.txt"
    with open(file, "r") as f:
        data = {}
        lines = f.readlines()
        option = 0
        for i, line in enumerate(lines):
            lines[i] = "".join([c for c in line if c not in ["\n",'']])
            if line != "\n":
                if not line.startswith(("PDB:", "#D", "A")):
                    res = lines[i].split("|")
                    boundaries = res[2]
                    domains = boundaries.strip().split(" ")
                    data[f"option{option}"] = {}
                    for j in range(len(domains)):
                        data[f"option{option}"][str(j+1)] = domains[j]
                    option += 1
    verb and pprint(data)
    return data

In [3]:
cath = get_cath()

In [40]:
def accuracy(d1, d2):
    # Accuracy = no of correct predictions / sequence size        
    seq_size = max(sequence_size(d1), sequence_size(d2))
    lst1 = domains_to_seq(d1, seq_size)
    lst2 = domains_to_seq(d2, seq_size)
    results = list(map(lambda x : x[0] == x[1], zip(lst1, lst2)))
    
    overlap = 0
    diff = 0
    for elt in results:
        if elt == True:
            overlap += 1
        else:
            diff += 1
            
    result = (overlap / (overlap + diff)) * 100
    # accuracy as percentage
    return float("{:.2f}".format(result)) 



def sequence_size(d):
    size = 0
    for value in d.values():
        value = int(value.split('-')[-1])
        if value > size:
            size = value
    return size


def compare(pdb, chain, f=accuracy, verb=True):
    accs = []
    for option in get_sword2(pdb, chain).keys():
        x = cath[pdb][chain]
        y = get_sword2(pdb, chain)[option]
        acc = f(x,y)
        verb and print(f"x: {x}\ny: {y}")
        verb and print(f"{option} accuracy: {acc}")
        accs.append((option, acc))
    return accs
    
    
def domains_to_seq(d, seq_size):
    no_of_domains = len(d.keys())
    lst = [0] * seq_size
    for i in range(1, no_of_domains + 1):
        domain = d.get(str(i), '0-0')
        for n in range(int(domain.split('-')[0])-1, int(domain.split('-')[-1])):
            lst[n] = i
    return lst

In [41]:
# pdbs = ['1a0h','1abe', '1acc', '1an9', '1b4v']
# pdb = '1an9'
# chain = 'A'
# for option in get_sword2(pdb, chain).keys():
#         x = cath[pdb][chain]
#         y = get_sword2(pdb, chain)[option]
#     #     print(f"x: {x}\ny: {y}")
#         acc = accuracy(y,x)
#         print(f"{option} accuracy: {acc}%")
    

In [42]:
# pdbs = ['1a0h','1abe', '1acc', '1an9', '1b4v']
pdbs = ['1a0h', '1an9', '1b4v']
chain = 'A'

In [68]:
# def casp6_acc(x, y):
    
#     target = [0] * len(x.keys())
#     predictions = [0] * len(y.keys())
    
#     seq_size = max(sequence_size(x), sequence_size(y))
#     lst_x = domains_to_seq(x, seq_size)
#     lst_y = domains_to_seq(y, seq_size)
    
#     for (t, p) in zip(lst_x, lst_y):
#         if t
            
#     result = 0
#     # accuracy as percentage
#     return float("{:.4f}".format(result)) * 100

In [110]:
def casp6_acc(x, y):
    
#     table = np.zeros((len(x.keys()) + 1, len(y.keys()) + 1))
    x_size = len(x.keys())
    y_size = len(y.keys())
    table = np.zeros((x_size, y_size))

    seq_size = max(sequence_size(x), sequence_size(y))
    lst_x = domains_to_seq(x, seq_size)
    lst_y = domains_to_seq(y, seq_size)
    for (t, p) in zip(lst_x, lst_y):
        if t == 0 or p == 0:
            continue
        table[t-1, p-1] += 1
            
    row_score = [0] * y_size
    col_score = [0] * x_size
    
    for i in range(y_size):
        for j in range(x_size):
            if i == j:
                row_score[i] += table[j, i]
                col_score[j] += table[j, i]
            else:
                row_score[i] -= table[j, i]
                col_score[j] -= table[j, i]
        
    
    score = (np.sum(row_score) + np.sum(col_score)) / 2
    normalized_score = score / sequence_size(x)
    # accuracy as percentage
    print("table")
    print(table)
    print("row")
    print(row_score)
    print("col")
    print(col_score)
    print("score")
    print(score)
    print("norm score")
    print(normalized_score)
#     return float("{:.4f}".format(result)) * 100


In [111]:
print("CASP6\n")
for pdb in pdbs:
    score = compare(pdb, chain, casp6_acc, False)
    print(pdb)
#     for (opt, y) in score:
#         print(f"{opt}:\n{y}")

CASP6

table
[[89.]
 [67.]]
row
[22.0]
col
[89.0, -67.0]
score
22.0
norm score
0.13836477987421383
table
[[52. 37.  0.]
 [11.  0. 56.]]
row
[41.0, -37.0, -56.0]
col
[15.0, -67.0]
score
-52.0
norm score
-0.3270440251572327
table
[[68. 21.  0.]
 [ 0. 11. 56.]]
row
[68.0, -10.0, -56.0]
col
[47.0, -45.0]
score
2.0
norm score
0.012578616352201259
table
[[89.  0.]
 [11. 56.]]
row
[78.0, 56.0]
col
[89.0, 45.0]
score
134.0
norm score
0.8427672955974843
table
[[68. 21.]
 [ 0. 67.]]
row
[68.0, 46.0]
col
[47.0, 67.0]
score
114.0
norm score
0.7169811320754716
1a0h
table
[[102.  40.]
 [  3. 194.]]
row
[99.0, 154.0]
col
[62.0, 191.0]
score
253.0
norm score
0.7463126843657817
table
[[48. 40.  0. 54.]
 [56. 52. 86.  3.]]
row
[-8.0, 12.0, -86.0, -57.0]
col
[-46.0, -93.0]
score
-139.0
norm score
-0.41002949852507375
table
[[48. 94.  0.  0.]
 [ 0. 61. 50. 86.]]
row
[48.0, -33.0, -50.0, -86.0]
col
[-46.0, -75.0]
score
-121.0
norm score
-0.35693215339233036
table
[[102.  40.   0.]
 [ 59.  52.  86.]]
row
[4

In [109]:
print("Accuracy\n")
for pdb in pdbs:
    score = compare(pdb, chain, accuracy, False)
    print(pdb)
    for (opt, y) in score:
        print(f"{opt}: {y}")

Accuracy

1a0h
option0: 55.97
option1: 32.7
option2: 49.69
option3: 91.19
option4: 84.91
1an9
option0: 87.06
option1: 29.41
option2: 32.06
option3: 45.29
option4: 46.76
option5: 67.06
1b4v
option0: 41.6
option1: 54.0
option2: 54.0
option3: 76.2
option4: 61.8
